In [1]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from pyproj import Proj, Transformer
import scipy
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### load function to convert lonlat to xy

In [2]:
def lonlat_to_xy(coords_1, coords_2, hemisphere, inverse=False):

    """Converts between longitude/latitude and EASE xy coordinates.
 
    Args:
        lon (float): WGS84 longitude
        lat (float): WGS84 latitude
        hemisphere (string): 'n' or 's'
        inverse (bool): if true, converts xy to lon/lat
 
    Returns:
        tuple: pair of xy or lon/lat values
    """

    EASE_Proj = {'n': 'EPSG:3408',
                 's': 'EPGS:3409'}
    
    WGS_Proj = 'EPSG:4326'
    
    for coords in [coords_1, coords_2]: assert isinstance(coords,(np.ndarray,list))

    if inverse == False: # lonlat to xy
        
        lon, lat = coords_1, coords_2
        
        transformer = Transformer.from_crs(WGS_Proj, EASE_Proj[hemisphere])
        
        x, y = transformer.transform(lat, lon)
        
        return (x, y)

    else: # xy to lonlat
        
        x, y = coords_1, coords_2
        
        transformer = Transformer.from_crs(EASE_Proj[hemisphere], WGS_Proj)
        
        lat, lon = transformer.transform(x, y)
        
        return (lon, lat)

### load buoy days and coordinates

In [ ]:
datapath = '/home/cjn/alpha_retracker/pysiral/'
buoy_locs = pd.read_csv(datapath+'locs.csv')
buoy_locs

### load function to calculate monthly average at each buoy 

In [4]:
def monthly_means(buoy):
    
    buoy_df=[]
    
    for threshold in np.arange(50,1000,25): ### loop through thresholds
        
        for year in np.arange(2010,2022): ### loop through years
            
            for month in np.arange(1,13): ### loop through months
                
                folder_path = f"{datapath_in}vthres{threshold:03d}/l2p/{year}/{month:02d}/"
                
                if os.path.exists(folder_path) == True: ### check if data exists
                    monthly_coords = buoy_locs[buoy_locs['month']==month][buoy_locs['year']==year][buoy_locs['mooring']==buoy]
                    
                    if len(monthly_coords)>0: ### check if buoy coords exist
                        buoy_lon = monthly_coords['lon'].iloc[0]
                        buoy_lat = monthly_coords['lat'].iloc[0]
                        buoy_x, buoy_y = lonlat_to_xy(np.array(buoy_lon),np.array(buoy_lat),hemisphere='n') 
                     
                        monthly_values = []
                        files = os.listdir(folder_path)

                        for file in files:
                            values = [] ### make list of all values
                            cs2_lons = np.array(Dataset(folder_path+file)['longitude'])
                            cs2_lats = np.array(Dataset(folder_path+file)['latitude'])
                            cs2_x,cs2_y = lonlat_to_xy(cs2_lons,cs2_lats,hemisphere='n')
                            KDTree = scipy.spatial.cKDTree(list(zip(cs2_x,cs2_y)))
                            hits = KDTree.query_ball_point(x=[buoy_x,buoy_y],r=radius*1000)

                            if len(hits)>0: ### if values within buoy radius
                                cs2_rf = np.array(Dataset(folder_path+file)['radar_freeboard'])
                                monthly_values.append(cs2_rf[hits]) ## append values for this track

                        if len(monthly_values)>0:
                            monthly_mean = np.nanmean(np.hstack(monthly_values)) ### take mean of all values
                            if ~np.isnan(monthly_mean):
                                buoy_df.append({"year":year,"month":month,"mean_radar_freeboard":monthly_mean,
                                                "threshold":threshold/10})

    mean_radar_freeboards = pd.DataFrame(buoy_df)
    mean_radar_freeboards.to_csv(f'{datapath_out}/mean_rf_mooring_{buoy}.csv', index=False)

### make dataframe of monthly buoy mean radar freeboards

In [ ]:
datapath_in = '/home/cn/cs2_alpha/DATA/cs2_l2/'
datapath_out = '/home/cjn/alpha_retracker/pysiral/'

radius = 25 ### km radius from buoy

for buoy in ['a','b','d','f11','f12','f13','f14']:

    monthly_means(buoy)